In [262]:
import subprocess, time, re
from termcolor import colored
from netaddr import IPAddress
import ipaddress

from passlib.utils import pbkdf2
import binascii

from prettytable import PrettyTable

In [263]:
ip_info=subprocess.run('ifconfig wlan0', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

In [264]:
print(ip_info.stdout)

wlan0: flags=4163<UP,BROADCAST,RUNNING,MULTICAST>  mtu 1500
        inet 192.168.50.117  netmask 255.255.192.0  broadcast 192.168.63.255
        inet6 fe80::473c:d558:928:d5c  prefixlen 64  scopeid 0x20<link>
        ether e4:5f:01:a4:5d:53  txqueuelen 1000  (Ethernet)
        RX packets 1155513  bytes 382128929 (364.4 MiB)
        RX errors 0  dropped 0  overruns 0  frame 0
        TX packets 50387  bytes 14965875 (14.2 MiB)
        TX errors 0  dropped 0 overruns 0  carrier 0  collisions 0




In [265]:
def get_ip_info(ip_info):
    '''
    Fetch ip address, subnet mask, gateway from ip_info
    '''
    ip_pattern = r'inet\s+((\d+\.){3}\d+)\s+'
    mask_pattern = r'netmask\s+((\d+\.){3}\d+)\s+'
    broadcast_pattern = r'broadcast\s+((\d+\.){3}\d+)'
    ip_match = re.search(ip_pattern, ip_info)
    mask_match = re.search(mask_pattern, ip_info)
    broadcast_match = re.search(broadcast_pattern, ip_info)
    if ip_match and mask_match and broadcast_match:
        ip = ip_match.group(1)
        mask = mask_match.group(1)
        broadcast = broadcast_match.group(1)
        return ip, mask, broadcast
    else:
        return None, None, None

In [266]:
ip, mask, broadcast= get_ip_info(ip_info.stdout)

In [267]:
print(ip, mask, broadcast)

192.168.50.117 255.255.192.0 192.168.63.255


In [268]:
# 取得 arp-scan 所有 ip
start_time = time.time()
arp_info=subprocess.run('sudo arp-scan -l --interface=wlan0', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
end_time = time.time()
print("cost time: ",end_time - start_time)

cost time:  66.0714340209961


In [269]:
print(arp_info.stdout)

Interface: wlan0, type: EN10MB, MAC: e4:5f:01:a4:5d:53, IPv4: 192.168.50.117
Starting arp-scan 1.9.7 with 16384 hosts (https://github.com/royhills/arp-scan)
192.168.18.58	b8:27:eb:fd:b3:f5	Raspberry Pi Foundation
192.168.19.1	b8:27:eb:bf:8b:2c	Raspberry Pi Foundation
192.168.19.24	dc:a6:32:ca:7c:7e	Raspberry Pi Trading Ltd
192.168.19.84	b8:27:eb:df:5c:00	Raspberry Pi Foundation
192.168.19.192	00:0e:c6:58:13:d8	ASIX ELECTRONICS CORP.
192.168.19.193	00:0e:c6:58:13:d8	ASIX ELECTRONICS CORP.
192.168.20.23	60:12:8b:19:ce:fa (0c:f5:a4:37:79:f0)	CANON INC.
192.168.20.41	54:4a:00:1e:7d:c0	Cisco Systems, Inc
192.168.20.109	dc:a6:32:57:c1:1f	Raspberry Pi Trading Ltd
192.168.21.56	88:1d:fc:46:d4:d0	Cisco Systems, Inc
192.168.27.80	20:0c:c8:15:e4:07 (0c:f5:a4:37:79:f0)	NETGEAR
192.168.30.20	60:12:8b:11:a4:f1 (0c:f5:a4:37:79:f0)	CANON INC.
192.168.30.22	74:bf:c0:bd:b0:71 (0c:f5:a4:37:79:f0)	CANON INC.
192.168.30.24	60:12:8b:2f:89:e5 (0c:f5:a4:37:79:f0)	CANON INC.
192.168.30.7	00:08:dc:16:bf:fd	Wizn

In [273]:
ip_pattern = r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b"

#只取ip填入列表的方法
ip_occupied = re.findall(ip_pattern, arp_info.stdout)
print(ip_occupied)

['192.168.50.117', '192.168.18.58', '192.168.19.1', '192.168.19.24', '192.168.19.84', '192.168.19.192', '192.168.19.193', '192.168.20.23', '192.168.20.41', '192.168.20.109', '192.168.21.56', '192.168.27.80', '192.168.30.20', '192.168.30.22', '192.168.30.24', '192.168.30.7', '192.168.30.10', '192.168.30.93', '192.168.30.88', '192.168.30.101', '192.168.30.103', '192.168.30.109', '192.168.30.157', '192.168.31.203', '192.168.45.2', '192.168.45.35', '192.168.45.82', '192.168.45.84', '192.168.45.107', '192.168.45.133', '192.168.45.151', '192.168.45.150', '192.168.45.177', '192.168.45.216', '192.168.45.245', '192.168.46.1', '192.168.46.46', '192.168.46.74', '192.168.46.67', '192.168.46.123', '192.168.46.122', '192.168.46.221', '192.168.46.233', '192.168.47.49', '192.168.47.57', '192.168.47.105', '192.168.47.117', '192.168.47.130', '192.168.47.164', '192.168.47.210', '192.168.47.221', '192.168.47.224', '192.168.47.250', '192.168.47.251', '192.168.48.0', '192.168.48.46', '192.168.48.66', '192.1

In [274]:

cidr = IPAddress(mask).netmask_bits()
print(cidr)

18


In [275]:

ip_cidr = ip + '/' + str(cidr)
print(ip_cidr)
local_net = ipaddress.ip_network(ip_cidr, strict=False)
print(local_net)


192.168.50.117/18
192.168.0.0/18


In [276]:
ip_count = 2**(32-cidr)-2
print(ip_count)

16382


In [277]:
ip_available = []
n = range(1,ip_count+1)
for i in n:
    if str(local_net[i]) in ip_occupied:
        text= '已占用: ' + str(local_net[i])
        print(colored(text,'red'))
    else:
        text= '可选用: ' + str(local_net[i])
        print(colored(text,'green'))
        ip_available.append(str(local_net[i]))
    

可选用: 192.168.0.1
可选用: 192.168.0.2
可选用: 192.168.0.3
可选用: 192.168.0.4
可选用: 192.168.0.5
可选用: 192.168.0.6
可选用: 192.168.0.7
可选用: 192.168.0.8
可选用: 192.168.0.9
可选用: 192.168.0.10
可选用: 192.168.0.11
可选用: 192.168.0.12
可选用: 192.168.0.13
可选用: 192.168.0.14
可选用: 192.168.0.15
可选用: 192.168.0.16
可选用: 192.168.0.17
可选用: 192.168.0.18
可选用: 192.168.0.19
可选用: 192.168.0.20
可选用: 192.168.0.21
可选用: 192.168.0.22
可选用: 192.168.0.23
可选用: 192.168.0.24
可选用: 192.168.0.25
可选用: 192.168.0.26
可选用: 192.168.0.27
可选用: 192.168.0.28
可选用: 192.168.0.29
可选用: 192.168.0.30
可选用: 192.168.0.31
可选用: 192.168.0.32
可选用: 192.168.0.33
可选用: 192.168.0.34
可选用: 192.168.0.35
可选用: 192.168.0.36
可选用: 192.168.0.37
可选用: 192.168.0.38
可选用: 192.168.0.39
可选用: 192.168.0.40
可选用: 192.168.0.41
可选用: 192.168.0.42
可选用: 192.168.0.43
可选用: 192.168.0.44
可选用: 192.168.0.45
可选用: 192.168.0.46
可选用: 192.168.0.47
可选用: 192.168.0.48
可选用: 192.168.0.49
可选用: 192.168.0.50
可选用: 192.168.0.51
可选用: 192.168.0.52
可选用: 192.168.0.53
可选用: 192.168.0.54
可选用: 192.168.0.55
可选用: 192.168.0.56
可

可选用: 192.168.3.191
可选用: 192.168.3.192
可选用: 192.168.3.193
可选用: 192.168.3.194
可选用: 192.168.3.195
可选用: 192.168.3.196
可选用: 192.168.3.197
可选用: 192.168.3.198
可选用: 192.168.3.199
可选用: 192.168.3.200
可选用: 192.168.3.201
可选用: 192.168.3.202
可选用: 192.168.3.203
可选用: 192.168.3.204
可选用: 192.168.3.205
可选用: 192.168.3.206
可选用: 192.168.3.207
可选用: 192.168.3.208
可选用: 192.168.3.209
可选用: 192.168.3.210
可选用: 192.168.3.211
可选用: 192.168.3.212
可选用: 192.168.3.213
可选用: 192.168.3.214
可选用: 192.168.3.215
可选用: 192.168.3.216
可选用: 192.168.3.217
可选用: 192.168.3.218
可选用: 192.168.3.219
可选用: 192.168.3.220
可选用: 192.168.3.221
可选用: 192.168.3.222
可选用: 192.168.3.223
可选用: 192.168.3.224
可选用: 192.168.3.225
可选用: 192.168.3.226
可选用: 192.168.3.227
可选用: 192.168.3.228
可选用: 192.168.3.229
可选用: 192.168.3.230
可选用: 192.168.3.231
可选用: 192.168.3.232
可选用: 192.168.3.233
可选用: 192.168.3.234
可选用: 192.168.3.235
可选用: 192.168.3.236
可选用: 192.168.3.237
可选用: 192.168.3.238
可选用: 192.168.3.239
可选用: 192.168.3.240
可选用: 192.168.3.241
可选用: 192.168.3.242
可选用: 192.168

可选用: 192.168.6.192
可选用: 192.168.6.193
可选用: 192.168.6.194
可选用: 192.168.6.195
可选用: 192.168.6.196
可选用: 192.168.6.197
可选用: 192.168.6.198
可选用: 192.168.6.199
可选用: 192.168.6.200
可选用: 192.168.6.201
可选用: 192.168.6.202
可选用: 192.168.6.203
可选用: 192.168.6.204
可选用: 192.168.6.205
可选用: 192.168.6.206
可选用: 192.168.6.207
可选用: 192.168.6.208
可选用: 192.168.6.209
可选用: 192.168.6.210
可选用: 192.168.6.211
可选用: 192.168.6.212
可选用: 192.168.6.213
可选用: 192.168.6.214
可选用: 192.168.6.215
可选用: 192.168.6.216
可选用: 192.168.6.217
可选用: 192.168.6.218
可选用: 192.168.6.219
可选用: 192.168.6.220
可选用: 192.168.6.221
可选用: 192.168.6.222
可选用: 192.168.6.223
可选用: 192.168.6.224
可选用: 192.168.6.225
可选用: 192.168.6.226
可选用: 192.168.6.227
可选用: 192.168.6.228
可选用: 192.168.6.229
可选用: 192.168.6.230
可选用: 192.168.6.231
可选用: 192.168.6.232
可选用: 192.168.6.233
可选用: 192.168.6.234
可选用: 192.168.6.235
可选用: 192.168.6.236
可选用: 192.168.6.237
可选用: 192.168.6.238
可选用: 192.168.6.239
可选用: 192.168.6.240
可选用: 192.168.6.241
可选用: 192.168.6.242
可选用: 192.168.6.243
可选用: 192.168

可选用: 192.168.10.62
可选用: 192.168.10.63
可选用: 192.168.10.64
可选用: 192.168.10.65
可选用: 192.168.10.66
可选用: 192.168.10.67
可选用: 192.168.10.68
可选用: 192.168.10.69
可选用: 192.168.10.70
可选用: 192.168.10.71
可选用: 192.168.10.72
可选用: 192.168.10.73
可选用: 192.168.10.74
可选用: 192.168.10.75
可选用: 192.168.10.76
可选用: 192.168.10.77
可选用: 192.168.10.78
可选用: 192.168.10.79
可选用: 192.168.10.80
可选用: 192.168.10.81
可选用: 192.168.10.82
可选用: 192.168.10.83
可选用: 192.168.10.84
可选用: 192.168.10.85
可选用: 192.168.10.86
可选用: 192.168.10.87
可选用: 192.168.10.88
可选用: 192.168.10.89
可选用: 192.168.10.90
可选用: 192.168.10.91
可选用: 192.168.10.92
可选用: 192.168.10.93
可选用: 192.168.10.94
可选用: 192.168.10.95
可选用: 192.168.10.96
可选用: 192.168.10.97
可选用: 192.168.10.98
可选用: 192.168.10.99
可选用: 192.168.10.100
可选用: 192.168.10.101
可选用: 192.168.10.102
可选用: 192.168.10.103
可选用: 192.168.10.104
可选用: 192.168.10.105
可选用: 192.168.10.106
可选用: 192.168.10.107
可选用: 192.168.10.108
可选用: 192.168.10.109
可选用: 192.168.10.110
可选用: 192.168.10.111
可选用: 192.168.10.112
可选用: 192.168.10.11

可选用: 192.168.13.190
可选用: 192.168.13.191
可选用: 192.168.13.192
可选用: 192.168.13.193
可选用: 192.168.13.194
可选用: 192.168.13.195
可选用: 192.168.13.196
可选用: 192.168.13.197
可选用: 192.168.13.198
可选用: 192.168.13.199
可选用: 192.168.13.200
可选用: 192.168.13.201
可选用: 192.168.13.202
可选用: 192.168.13.203
可选用: 192.168.13.204
可选用: 192.168.13.205
可选用: 192.168.13.206
可选用: 192.168.13.207
可选用: 192.168.13.208
可选用: 192.168.13.209
可选用: 192.168.13.210
可选用: 192.168.13.211
可选用: 192.168.13.212
可选用: 192.168.13.213
可选用: 192.168.13.214
可选用: 192.168.13.215
可选用: 192.168.13.216
可选用: 192.168.13.217
可选用: 192.168.13.218
可选用: 192.168.13.219
可选用: 192.168.13.220
可选用: 192.168.13.221
可选用: 192.168.13.222
可选用: 192.168.13.223
可选用: 192.168.13.224
可选用: 192.168.13.225
可选用: 192.168.13.226
可选用: 192.168.13.227
可选用: 192.168.13.228
可选用: 192.168.13.229
可选用: 192.168.13.230
可选用: 192.168.13.231
可选用: 192.168.13.232
可选用: 192.168.13.233
可选用: 192.168.13.234
可选用: 192.168.13.235
可选用: 192.168.13.236
可选用: 192.168.13.237
可选用: 192.168.13.238
可选用: 192.168.13.239


...

已占用: 192.168.49.46
可选用: 192.168.49.47
已占用: 192.168.49.48
已占用: 192.168.49.49
可选用: 192.168.49.50
已占用: 192.168.49.51
可选用: 192.168.49.52
可选用: 192.168.49.53
已占用: 192.168.49.54
可选用: 192.168.49.55
已占用: 192.168.49.56
可选用: 192.168.49.57
可选用: 192.168.49.58
已占用: 192.168.49.59
可选用: 192.168.49.60
可选用: 192.168.49.61
可选用: 192.168.49.62
可选用: 192.168.49.63
可选用: 192.168.49.64
可选用: 192.168.49.65
已占用: 192.168.49.66
已占用: 192.168.49.67
已占用: 192.168.49.68
已占用: 192.168.49.69
已占用: 192.168.49.70
已占用: 192.168.49.71
可选用: 192.168.49.72
已占用: 192.168.49.73
已占用: 192.168.49.74
已占用: 192.168.49.75
可选用: 192.168.49.76
已占用: 192.168.49.77
可选用: 192.168.49.78
已占用: 192.168.49.79
已占用: 192.168.49.80
可选用: 192.168.49.81
已占用: 192.168.49.82
已占用: 192.168.49.83
已占用: 192.168.49.84
可选用: 192.168.49.85
可选用: 192.168.49.86
可选用: 192.168.49.87
可选用: 192.168.49.88
已占用: 192.168.49.89
可选用: 192.168.49.90
可选用: 192.168.49.91
可选用: 192.168.49.92
可选用: 192.168.49.93
已占用: 192.168.49.94
可选用: 192.168.49.95
可选用: 192.168.49.96
已占用: 192.168.49.97
已占用: 192.168

可选用: 192.168.51.184
可选用: 192.168.51.185
可选用: 192.168.51.186
可选用: 192.168.51.187
可选用: 192.168.51.188
可选用: 192.168.51.189
可选用: 192.168.51.190
可选用: 192.168.51.191
可选用: 192.168.51.192
可选用: 192.168.51.193
可选用: 192.168.51.194
可选用: 192.168.51.195
可选用: 192.168.51.196
可选用: 192.168.51.197
可选用: 192.168.51.198
可选用: 192.168.51.199
可选用: 192.168.51.200
可选用: 192.168.51.201
可选用: 192.168.51.202
可选用: 192.168.51.203
可选用: 192.168.51.204
可选用: 192.168.51.205
可选用: 192.168.51.206
可选用: 192.168.51.207
可选用: 192.168.51.208
可选用: 192.168.51.209
可选用: 192.168.51.210
可选用: 192.168.51.211
可选用: 192.168.51.212
可选用: 192.168.51.213
可选用: 192.168.51.214
可选用: 192.168.51.215
可选用: 192.168.51.216
可选用: 192.168.51.217
可选用: 192.168.51.218
可选用: 192.168.51.219
可选用: 192.168.51.220
可选用: 192.168.51.221
可选用: 192.168.51.222
可选用: 192.168.51.223
可选用: 192.168.51.224
可选用: 192.168.51.225
可选用: 192.168.51.226
可选用: 192.168.51.227
可选用: 192.168.51.228
可选用: 192.168.51.229
可选用: 192.168.51.230
可选用: 192.168.51.231
可选用: 192.168.51.232
可选用: 192.168.51.233


可选用: 192.168.55.56
可选用: 192.168.55.57
可选用: 192.168.55.58
可选用: 192.168.55.59
可选用: 192.168.55.60
可选用: 192.168.55.61
可选用: 192.168.55.62
可选用: 192.168.55.63
可选用: 192.168.55.64
可选用: 192.168.55.65
可选用: 192.168.55.66
可选用: 192.168.55.67
可选用: 192.168.55.68
可选用: 192.168.55.69
可选用: 192.168.55.70
可选用: 192.168.55.71
可选用: 192.168.55.72
可选用: 192.168.55.73
可选用: 192.168.55.74
可选用: 192.168.55.75
可选用: 192.168.55.76
可选用: 192.168.55.77
可选用: 192.168.55.78
可选用: 192.168.55.79
可选用: 192.168.55.80
可选用: 192.168.55.81
可选用: 192.168.55.82
可选用: 192.168.55.83
可选用: 192.168.55.84
可选用: 192.168.55.85
可选用: 192.168.55.86
可选用: 192.168.55.87
可选用: 192.168.55.88
可选用: 192.168.55.89
可选用: 192.168.55.90
可选用: 192.168.55.91
可选用: 192.168.55.92
可选用: 192.168.55.93
可选用: 192.168.55.94
可选用: 192.168.55.95
可选用: 192.168.55.96
可选用: 192.168.55.97
可选用: 192.168.55.98
可选用: 192.168.55.99
已占用: 192.168.55.100
可选用: 192.168.55.101
可选用: 192.168.55.102
可选用: 192.168.55.103
可选用: 192.168.55.104
可选用: 192.168.55.105
可选用: 192.168.55.106
已占用: 192.168.55.107
可选用:

可选用: 192.168.59.55
可选用: 192.168.59.56
可选用: 192.168.59.57
可选用: 192.168.59.58
可选用: 192.168.59.59
可选用: 192.168.59.60
可选用: 192.168.59.61
可选用: 192.168.59.62
可选用: 192.168.59.63
可选用: 192.168.59.64
可选用: 192.168.59.65
可选用: 192.168.59.66
可选用: 192.168.59.67
可选用: 192.168.59.68
可选用: 192.168.59.69
可选用: 192.168.59.70
可选用: 192.168.59.71
可选用: 192.168.59.72
可选用: 192.168.59.73
可选用: 192.168.59.74
可选用: 192.168.59.75
可选用: 192.168.59.76
可选用: 192.168.59.77
可选用: 192.168.59.78
可选用: 192.168.59.79
可选用: 192.168.59.80
可选用: 192.168.59.81
可选用: 192.168.59.82
可选用: 192.168.59.83
可选用: 192.168.59.84
可选用: 192.168.59.85
可选用: 192.168.59.86
可选用: 192.168.59.87
可选用: 192.168.59.88
可选用: 192.168.59.89
可选用: 192.168.59.90
可选用: 192.168.59.91
可选用: 192.168.59.92
可选用: 192.168.59.93
可选用: 192.168.59.94
可选用: 192.168.59.95
可选用: 192.168.59.96
可选用: 192.168.59.97
可选用: 192.168.59.98
可选用: 192.168.59.99
可选用: 192.168.59.100
可选用: 192.168.59.101
可选用: 192.168.59.102
可选用: 192.168.59.103
可选用: 192.168.59.104
可选用: 192.168.59.105
可选用: 192.168.59.106
可选用: 

可选用: 192.168.62.54
可选用: 192.168.62.55
可选用: 192.168.62.56
可选用: 192.168.62.57
可选用: 192.168.62.58
可选用: 192.168.62.59
可选用: 192.168.62.60
可选用: 192.168.62.61
可选用: 192.168.62.62
可选用: 192.168.62.63
可选用: 192.168.62.64
可选用: 192.168.62.65
可选用: 192.168.62.66
可选用: 192.168.62.67
可选用: 192.168.62.68
可选用: 192.168.62.69
可选用: 192.168.62.70
可选用: 192.168.62.71
可选用: 192.168.62.72
可选用: 192.168.62.73
可选用: 192.168.62.74
可选用: 192.168.62.75
可选用: 192.168.62.76
可选用: 192.168.62.77
可选用: 192.168.62.78
可选用: 192.168.62.79
可选用: 192.168.62.80
可选用: 192.168.62.81
可选用: 192.168.62.82
可选用: 192.168.62.83
可选用: 192.168.62.84
可选用: 192.168.62.85
可选用: 192.168.62.86
可选用: 192.168.62.87
可选用: 192.168.62.88
可选用: 192.168.62.89
可选用: 192.168.62.90
可选用: 192.168.62.91
可选用: 192.168.62.92
可选用: 192.168.62.93
可选用: 192.168.62.94
可选用: 192.168.62.95
可选用: 192.168.62.96
可选用: 192.168.62.97
可选用: 192.168.62.98
可选用: 192.168.62.99
可选用: 192.168.62.100
可选用: 192.168.62.101
可选用: 192.168.62.102
可选用: 192.168.62.103
可选用: 192.168.62.104
可选用: 192.168.62.105
可选用: 1

In [278]:
# print(ip_available)

In [279]:

SSID="Ashcloud"
WPA_SECRET="Passw0rd@1213|0412"
WPA_PSK_KEY = pbkdf2.pbkdf2(str.encode(WPA_SECRET), str.encode(SSID), 4096, 32)
WPA_PSK_RAWKEY = binascii.hexlify(WPA_PSK_KEY).decode("utf-8").upper()
print(WPA_PSK_RAWKEY)


D8AEB3EA544076EEB5824816891DEDD92115932A672E975F9E35B7A754DB7220


In [280]:
D8AEB3EA544076EEB5824816891DEDD92115932A672E975F9E35B7A754DB7220

NameError: name 'D8AEB3EA544076EEB5824816891DEDD92115932A672E975F9E35B7A754DB7220' is not defined

In [281]:
ssid = "Ashcloud"

In [282]:
template = f"""
^XA
^WIA
^NC2
^NPP
^KC1,0,,
^WAD,D
^WEOFF,1,,,,,,
^WP0,0
^WR,,,,100
^WS{ssid},I,L,,,
^NBS
^WLOFF,,
^WKOFF,,,,
^WX09,{WPA_PSK_RAWKEY}
^XZ
^XA
^JUS
^XZ
! U1 setvar "wlan.country_code" "china"
! U1 setvar "wlan.international_mode" "off"
! U1 setvar "wlan.allowed_band" "all"
! U1 do "device.reset" ""
"""

In [283]:
print(template)


^XA
^WIA
^NC2
^NPP
^KC1,0,,
^WAD,D
^WEOFF,1,,,,,,
^WP0,0
^WR,,,,100
^WSAshcloud,I,L,,,
^NBS
^WLOFF,,
^WKOFF,,,,
^WX09,D8AEB3EA544076EEB5824816891DEDD92115932A672E975F9E35B7A754DB7220
^XZ
^XA
^JUS
^XZ
! U1 setvar "wlan.country_code" "china"
! U1 setvar "wlan.international_mode" "off"
! U1 setvar "wlan.allowed_band" "all"
! U1 do "device.reset" ""



In [284]:
#open text file
text_file = open("./wifi.zpl", "w")
 
#write string to file
n = text_file.write(template)
 
#close file
text_file.close()
 
print(n)

349


In [286]:
usb_cmd=subprocess.run('sudo echo wifi.zpl > /dev/usb/lp0', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("out: ", usb_cmd.stdout)
print("error: ", usb_cmd.stderr)

out:  
error:  /bin/sh: 1: cannot create /dev/usb/lp0: Permission denied



In [288]:
print("请在上面选择一个绿色可用IP填入,请尽量在绿色连续区中挑选,避免以后IP冲突")
while True:
    printer_ip = input("输入IP: ")
    if printer_ip not in ip_available:
        print("输入不正确！请在上面选择一个绿色可用IP填入!")
    else:
        break
print(colored(f"选定IP为: {printer_ip}",'green'))
print(colored("请将IP标记在此打印机上！", 'red'))

请在上面选择一个绿色可用IP填入,请尽量在绿色连续区中挑选,避免以后IP冲突
输入IP: 192.168.48.1
选定IP为: 192.168.48.1
请将IP标记在此打印机上！


In [289]:
wireless=subprocess.run("sudo iwlist wlan0 scan | grep 'ESSID\|Cell\|Frequency\|Authentication\|Quality\|Channel\|Encryption'", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

print(wireless.stdout)

          Cell 01 - Address: E0:D1:73:28:85:12
                    Channel:1
                    Frequency:2.412 GHz (Channel 1)
                    Quality=46/70  Signal level=-64 dBm  
                    Encryption key:on
                    ESSID:"test"
                        Authentication Suites (1) : PSK
                        Authentication Suites (1) : PSK
          Cell 02 - Address: 00:68:EB:1E:C9:B4
                    Channel:6
                    Frequency:2.437 GHz (Channel 6)
                    Quality=57/70  Signal level=-53 dBm  
                    Encryption key:on
                    ESSID:"DIRECT-B3-HP DeskJet 2600 series"
                        Authentication Suites (1) : PSK
          Cell 03 - Address: 0C:F5:A4:37:79:FF
                    Channel:165
                    Frequency:5.825 GHz (Channel 165)
                    Quality=55/70  Signal level=-55 dBm  
                    Encryption key:on
                    ESSID:"AshCloud"
                      

In [290]:
wifi_list=wireless.stdout.split('Cell ')[1:]

In [291]:
print(wifi_list[2])

03 - Address: 0C:F5:A4:37:79:FF
                    Channel:165
                    Frequency:5.825 GHz (Channel 165)
                    Quality=55/70  Signal level=-55 dBm  
                    Encryption key:on
                    ESSID:"AshCloud"
                        Authentication Suites (1) : PSK
                        Authentication Suites (1) : PSK
          


In [293]:
print(frequency.group(1))
print(frequency.group())

2.462
Frequency:2.462


In [294]:
"PSK" in wifi_list[2]

True

In [295]:

print(quality.group(1))

33/70


In [296]:

print(channel.group(1))

11


In [297]:
address_pattern = r"([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})"
address = re.search(address_pattern, wifi_list[0])
print(address.group())

E0:D1:73:28:85:12


In [298]:

print(encrypted.group(1))

on


In [299]:
len(wifi_list)

13

In [300]:
#regex patterns 

essid_pattern = r'ESSID:\"(.*)"'
freq_pattern = r"Frequency:(\d.\d+)"
quality_pattern = r"Quality=(\d+/\d+)"
channel_pattern = r"Channel:(\d+)"
address_pattern = r"(([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2}))"
encrypted_pattern = r"Encryption key:(\w+)"
rssi_pattern = r"Signal level=(-\d+)"



In [301]:
def get_group1(matched):
    """
    return the group(1) of regex obj or N/A if not matched
    """
    if matched:
        return matched.group(1)
    else:
        return "N/A"

In [302]:
wifi_dicts = []

for w in range(len(wifi_list)):
    
    ssid = re.search(essid_pattern, wifi_list[w])
    frequency = re.search(freq_pattern, wifi_list[w])
    quality = re.search(quality_pattern, wifi_list[w])
    channel = re.search(channel_pattern, wifi_list[w])
    address = re.search(address_pattern, wifi_list[w])
    encrypted = re.search(encrypted_pattern, wifi_list[w])
    rssi = re.search(rssi_pattern, wifi_list[w])
    psk = "PSK" in wifi_list[w]
    
    freq = get_group1(frequency)[0:1]
    if freq == '2':
        band = '2.4 Ghz'
    elif freq == '5':
        band = '5 Ghz'
    else:
        band = 'N/A'
    
    wifi_dicts.append({
        "id" : w+1,
        "ssid" : get_group1(ssid),
        "frequency" : get_group1(frequency),
        "quality" : get_group1(quality),
        "channel" : get_group1(channel),
        "bssid" : get_group1(address),
        "encrypted" : get_group1(encrypted),
        "rssi" : get_group1(rssi),
        "psk" : psk,
        "band" : band

    })
print(wifi_dicts)

[{'id': 1, 'ssid': 'test', 'frequency': '2.412', 'quality': '46/70', 'channel': '1', 'bssid': 'E0:D1:73:28:85:12', 'encrypted': 'on', 'rssi': '-64', 'psk': True, 'band': '2.4 Ghz'}, {'id': 2, 'ssid': 'DIRECT-B3-HP DeskJet 2600 series', 'frequency': '2.437', 'quality': '57/70', 'channel': '6', 'bssid': '00:68:EB:1E:C9:B4', 'encrypted': 'on', 'rssi': '-53', 'psk': True, 'band': '2.4 Ghz'}, {'id': 3, 'ssid': 'AshCloud', 'frequency': '5.825', 'quality': '55/70', 'channel': '165', 'bssid': '0C:F5:A4:37:79:FF', 'encrypted': 'on', 'rssi': '-55', 'psk': True, 'band': '5 Ghz'}, {'id': 4, 'ssid': 'AshCloud', 'frequency': '2.412', 'quality': '47/70', 'channel': '1', 'bssid': 'E0:D1:73:28:85:10', 'encrypted': 'on', 'rssi': '-63', 'psk': True, 'band': '2.4 Ghz'}, {'id': 5, 'ssid': 'AshCloud', 'frequency': '2.437', 'quality': '51/70', 'channel': '6', 'bssid': 'D4:C9:3C:1F:4D:D0', 'encrypted': 'on', 'rssi': '-59', 'psk': True, 'band': '2.4 Ghz'}, {'id': 6, 'ssid': 'test', 'frequency': '2.437', 'quali

In [303]:
def regex_search(pattern, wifi_data):
    try:
        result = re.search(pattern, wifi_data)
    except:
        print("E")
        return None
    finally:
        return result

In [304]:
a = regex_search(essid_pattern, 'ESSID:"888"')

In [305]:
address_pattern = r"\b(([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2}))\b"
address = re.search(address_pattern, "07 - Address: 0C:F5:A4:37:79:FE")


In [306]:
get_re_group1(address)

'0C:F5:A4:37:79:FE'

In [319]:
def get_curretn_ssid(interface):
    """
    取得当前SSID
    如获取错误返回 "N/A"
    """
    raw=subprocess.run('iwgetid '+interface, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    # 如果取值异常，直接返回 "N/A"
    if raw.stderr:
        return "N/A"
    print(raw.stdout)
    
    # 正则式取值
    essid_pattern = r'ESSID:\"(.*)"'
    current_ssid_re = re.search(essid_pattern, raw.stdout)
    
    # if 取值失败返回 "N/A"
    if current_ssid_re:
        current_ssid = get_group1(current_ssid_re)
    else:
        current_ssid = "N/A"
    
    return current_ssid

print(get_curretn_ssid('wlan0'))


N/A


In [323]:
def print_wifi_table(wifi_dicts):
    """
    Wifi信息，表格显示出来
    """
    t = PrettyTable(['ID','SSID', 'RSSI', 'BAND', 'Ch.', 'Enc.', 'PSK', 'BSSID'])
    for row in wifi_dicts:
        t.add_row([row['id'],
                   row['ssid'],
                   row['rssi'],
                   row['band'],
                   row['channel'],
                   row['encrypted'],
                   row['psk'],
                   row['bssid']
                  ])

    t.align["ID"] = "r"    
    t.align["SSID"] = "l"
    t.align["BAND"] = "r"
    t.align['Ch.'] = "r"

    print(t)
    
print_wifi_table(wifi_dicts)

+----+----------------------------------+------+---------+-----+------+------+-------------------+
| ID | SSID                             | RSSI |    BAND | Ch. | Enc. | PSK  |       BSSID       |
+----+----------------------------------+------+---------+-----+------+------+-------------------+
|  1 | test                             | -64  | 2.4 Ghz |   1 |  on  | True | E0:D1:73:28:85:12 |
|  2 | DIRECT-B3-HP DeskJet 2600 series | -53  | 2.4 Ghz |   6 |  on  | True | 00:68:EB:1E:C9:B4 |
|  3 | AshCloud                         | -55  |   5 Ghz | 165 |  on  | True | 0C:F5:A4:37:79:FF |
|  4 | AshCloud                         | -63  | 2.4 Ghz |   1 |  on  | True | E0:D1:73:28:85:10 |
|  5 | AshCloud                         | -59  | 2.4 Ghz |   6 |  on  | True | D4:C9:3C:1F:4D:D0 |
|  6 | test                             | -60  | 2.4 Ghz |   6 |  on  | True | D4:C9:3C:1F:4D:D2 |
|  7 | AshCloud                         | -66  | 2.4 Ghz |  11 |  on  | True | 0C:F5:A4:37:79:F0 |
|  8 | tes